In [1]:
# Anathi Hlaula
### The Objective: Fine Tuning for better results 

In [2]:
# 1. Data Preparation 

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define paths
data_dir = 'Data'  # Path to the extracted data

train_dir = os.path.join(data_dir, 'Train')
valid_dir = os.path.join(data_dir, 'Valid')
test_dir = os.path.join(data_dir, 'Test')

# Image data generator with data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224),
                                                  batch_size=32, class_mode='categorical')

Found 610 images belonging to 5 classes.
Found 72 images belonging to 5 classes.
Found 315 images belonging to 5 classes.


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
import matplotlib.pyplot as plt
import os

# Define the number of classes in your dataset
num_classes = 5  # Replace with the correct number of classes

# Define the hypermodel
class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = models.Sequential()
        model.add(layers.Conv2D(
            filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
            activation='relu',
            input_shape=(150, 150, 3)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(
            units=hp.Int('units', min_value=32, max_value=128, step=32),
            activation='relu'))
        model.add(layers.Dense(num_classes, activation='softmax'))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                hp.Choice('learning_rate', values=[1e-3, 1e-4])),
            loss='categorical_crossentropy',
            metrics=['accuracy'])
        return model

# Instantiate the tuner
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='cnn_tuning')

# Define the data generators
Train_datagen = ImageDataGenerator(rescale=1./255)
Train_generator = Train_datagen.flow_from_directory(
    'Data/Train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'Data/validation', 'Data/Train ,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Perform hyperparameter tuning
tuner.search(Train_generator, epochs=10, validation_data=validation_generator)

SyntaxError: unterminated string literal (detected at line 53) (953045677.py, line 53)

In [14]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the correct paths to your data directories
train_data_dir = 'data/train'
valid_data_dir = 'data/validation'

# Load and preprocess data
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Define the model builder function
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(150, 150, 3)
    ))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(keras.layers.Dense(5, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the tuner
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Define early stopping
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Perform hyperparameter tuning
tuner.search(train_generator, epochs=10, validation_data=valid_generator, callbacks=[stop_early])

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of filters is {best_hps.get('filters')},
the optimal kernel size is {best_hps.get('kernel_size')}, the optimal number of units in the dense layer is {best_hps.get('units')},
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# Train the best model
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_generator, epochs=10, validation_data=valid_generator)

# Evaluate the model
val_loss, val_acc = model.evaluate(valid_generator)
print(f"Validation accuracy: {val_acc}")
print(f"Validation loss: {val_loss}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/train'

In [ ]:
# 2. Model Building 

In [ ]:
!pip install keras-tuner


In [ ]:
import kerastuner as kt

In [ ]:
from tensorflow.keras import layers, models
from kerastuner import HyperModel

class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = models.Sequential()
        
        model.add(layers.Conv2D(filters=hp.Int('filters_1', 32, 128, step=32), kernel_size=hp.Choice('kernel_size_1', [3, 5]),
                                activation=hp.Choice('activation_1', ['relu', 'tanh']), input_shape=(224, 224, 3)))
        model.add(layers.MaxPooling2D(pool_size=2))
        
        for i in range(hp.Int('conv_layers', 1, 3)):
            model.add(layers.Conv2D(filters=hp.Int(f'filters_{i+2}', 32, 128, step=32), kernel_size=hp.Choice(f'kernel_size_{i+2}', [3, 5]),
                                    activation=hp.Choice(f'activation_{i+2}', ['relu', 'tanh'])))
            model.add(layers.MaxPooling2D(pool_size=2))
        
        model.add(layers.Flatten())
        
        for i in range(hp.Int('dense_layers', 1, 3)):
            model.add(layers.Dense(units=hp.Int(f'units_{i+1}', 32, 128, step=32), activation=hp.Choice(f'dense_activation_{i+1}', ['relu', 'tanh'])))
        
        model.add(layers.Dense(4, activation='softmax'))
        
        model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model

In [ ]:
# 3. Hyperparameter Tuning

In [ ]:
from kerastuner.tuners import RandomSearch

# Define the tuner
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='hyperparam_tuning',
    project_name='cancer_classification'
)

# Perform hyperparameter tuning
tuner.search(train_generator, epochs=10, validation_data=valid_generator)

In [ ]:
# 4. Training the Model

In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
history = model.fit(train_generator, epochs=20, validation_data=valid_generator)

In [ ]:
# 5. Evaluation and Plotting

In [ ]:
import matplotlib.pyplot as plt

# Plot the training and validation accuracy
plt.figure(figsize=(12, 8))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()